In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as colors
from matplotlib.colors import BoundaryNorm
import cmocean 
import numpy as np
import numpy.ma as ma
import os 
import xarray as xr
import pandas as pd
import datetime
import warnings
warnings.filterwarnings("ignore")
import pyproj
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cpf
from cartopy.feature import NaturalEarthFeature, LAND, COASTLINE
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
from datetime import datetime, timedelta

In [1]:
import xarray as xr

OMBraw = xr.open_mfdataset('data/cruise/2024_KVS_deployment.nc')

crrt_OMB = "KVS-12"

OMB_instrument = OMBraw.sel(trajectory=crrt_OMB)

OMB_instrument.time[1000].values


/home/maltem/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


numpy.datetime64('2024-05-05T17:30:38.000000000')

In [2]:
OMBraw.trajectory

<xarray.DataArray 'trajectory' (trajectory: 33)>
array(['KVS-01', 'KVS-02', 'KVS-03', 'KVS-04', 'KVS-06', 'KVS-07', 'KVS-08',
       'KVS-09', 'KVS-10', 'KVS-11', 'KVS-12', 'KVS-13', 'KVS-14', 'KVS-15',
       'KVS-16', 'KVS-17', 'KVS-18', 'KVS-19', 'KVS-20', 'KVS-21', 'KVS-22',
       'KVS-23', 'KVS-24', 'KVS-25', 'KVS-26', 'KVS-27', 'KVS-28', 'KVS-29',
       'KVS-30', 'KVS-31', 'KVS-32', 'KVS-34', 'KVS-35'], dtype=object)
Coordinates:
  * trajectory  (trajectory) object 'KVS-01' 'KVS-02' ... 'KVS-34' 'KVS-35'
Attributes:
    cf_role:        trajectory_id
    standard_name:  platform_id

In [ ]:
def findindex(alat,alon,plat,plon):
    #finding identical location of pos plat, plon in array alat[],alon[]
    abslat = np.abs(alat-plat)
    abslon = np.abs(alon-plon)
    c = np.maximum(abslon,abslat)
    #latlon_idx = np.argmin(c)
    x, y = np.where(c == np.min(c))
    #print(alats[x,y],alon[x,y])
    x=int(x)
    y=int(y)
    return (x,y)

In [ ]:
# Dummy AROME Arctic GRID
arch_dir='../../python/data/basicAA/'
GRIDdummy = xr.open_mfdataset(arch_dir+'mean.diff.SFX_ICE_THK.basic_aa_new-amsr2.Mar2019.nc')
alat=np.array(GRIDdummy['latitude'])
alon=np.array(GRIDdummy['longitude'])

#Input AROME Arctic File reduced to MIZ
AAbox = xr.open_mfdataset('data/AromeArctic/box.fc.diag.svalmiz2024.nc')
aboxlat=np.array(AAbox['latitude'])
aboxlon=np.array(AAbox['longitude'])
#Input OMB data
OMBraw = xr.open_mfdataset('data/cruise/2024_KVS_deployment.nc')

In [ ]:
# crrt_time = datetime.datetime(2024, 4, 14, 12, 30, 0)
crrt_OMB = "KVS-12"

OMB_instrument = OMBraw.sel(trajectory=crrt_OMB)

OMB_instrument.time[1000].values

In [ ]:
# crrt_time = datetime.datetime(2024, 4, 14, 12, 30, 0)
crrt_OMB = "KVS-06"

OMB_instrument = OMBraw.sel(trajectory=crrt_OMB)

# Loop over the time axes of the temperature records and find corresponding positions lat/lon
OMBlat=[]; OMBlon=[]

for t in range(len(OMB_instrument.time_temp)):
    tx = np.argmin(np.abs(np.array(OMB_instrument.time-OMB_instrument.time_temp[t])))
    OMBlat.append(OMB_instrument.lat[tx])
    OMBlon.append(OMB_instrument.lon[tx])
    
#ix, jx = findindex(aboxlat,aboxlon,np.float32(crrt_lat),np.float32(crrt_lon))
#AAbox.interp(time=crrt_time).air_temperature_2m[0,ix,jx]    

In [ ]:

AAt2m=[];AAt0m=[]
for i in range(len(OMB_instrument.time_temp)):
    if (~np.isnan(np.array(OMBlat[i]))):
        ix, jx = findindex(aboxlat,aboxlon,np.float32(OMBlat[i]),np.float32(OMBlon[i]))
        tx = np.argmin(np.abs(np.array(AAbox.time-OMB_instrument.time_temp[i])))
        AAt2m.append(AAbox.air_temperature_2m[tx,0,ix,jx]-273.15)
        AAt0m.append(AAbox.air_temperature_0m[tx,0,ix,jx]-273.15)
    else:
        AAt2m.append(np.nan)
        AAt0m.append(np.nan)

In [ ]:
start_time = pd.to_datetime('2024-04-05 15:00')
end_time   = pd.to_datetime('2024-04-17 23:00')


plt.figure(figsize=(12, 8))
plt.plot(OMB_instrument.time_temp,OMB_instrument.temp_1m_calibrated, marker='.', linestyle='-', label='Obs 1m-temp')
#plt.plot(OMB_instrument.time_temp,OMB_instrument.temp_10cm_calibrated, marker='.', linestyle='-', label='Obs 10cm-temp')

plt.plot(OMB_instrument.time_temp,AAt2m, marker='.', linestyle='--', label='AROME Arctic 2m-temp')
plt.plot(OMB_instrument.time_temp,AAt0m, marker='.', linestyle='--', label='AROME Arctic 0m-temp')

plt.title('Air temperature  - '+ crrt_OMB,fontsize=17)
plt.xlabel('Date')
plt.ylabel('Temperature (°C)',fontsize=14)
plt.grid(True)
plt.xticks(rotation=45)
plt.legend(fontsize=14)
plt.tight_layout()
# Formatierung der X-Achse
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=12))
# Anzeigen des Plots
#plt.xlim(start_time, end_time)
#plt.ylim([-25,-8])
plt.show()

In [ ]:
map_proj = ccrs.LambertConformal(central_latitude=77.5,
                                 central_longitude=-25,
                                 standard_parallels=(77.5,77.5))
PRJ = pyproj.Proj(map_proj.proj4_init)
x0,y0 = PRJ(alon[460,20],alat[460,20])
x1,y1 = PRJ(alon[610,230],alat[610,230])
LAND_highres = cpf.NaturalEarthFeature('physical', 'land', '50m', edgecolor='face', \
                                       facecolor = 'darkgrey', linewidth=.1)
sizefont = 18
ice_cmap = colors.ListedColormap(['none','grey'])
ice_levels = np.linspace(0,1,3)
ice_norm = BoundaryNorm(ice_levels, 31)
xticks = [-20,-10, -8,-6,-4,-2,0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,50,60,70,80,90,100,110,120]
yticks = [60,65,70, 75,76,77,78,79,80,81,82,83, 85,90]

#fig, axs = plt.subplots(1,1, figsize=(8,8), facecolor='w', edgecolor='k')
# Create a figure with two subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 8), subplot_kw={'projection': map_proj})

fig.subplots_adjust(hspace = 0.1, wspace=0.3)

levels = np.linspace(0.,100 , 31)  # color bar
lticks =np.arange(0, 100+10, 10)
norm = BoundaryNorm(levels, 256)
colorscale = 'cmo.ice'

# (a)  -------

ax1.set_title('Air temperature deviation', fontsize = sizefont)

for sx in range(len(OMBlat)):
    scatter = ax1.scatter(OMBlon[sx],OMBlat[sx],c=OMB_instrument.temp_1m_calibrated[sx]-AAt2m[sx],s=50,cmap=cmocean.cm.balance,vmin=-6,vmax=6,transform = ccrs.PlateCarree(),alpha=0.8)

# Add color bar
cb = plt.colorbar(scatter, ax=ax1, orientation='horizontal', extend='both')
cb.set_label('in K', size=14)
cb.ax.tick_params(labelsize=12)
ax1.add_feature(LAND_highres, zorder = 1, alpha = 1.0)
ax1.set_xlim(x0,x1); ax1.set_ylim(y0,y1)
# Make Lambert Grid Lines:
fig.canvas.draw() # must call this for labeling to work properly on outer boundary of figure.
ax1.gridlines(xlocs=xticks, ylocs=yticks,color='dimgrey')
ax1.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax1.yaxis.set_major_formatter(LATITUDE_FORMATTER)

#fig.savefig('icethickness.png')
